In [ ]:
from matplotlib import cm
from itertools import product

In [ ]:
var('r,t,theta,phi,x,y,z')
assume(x,'real')
assume(y,'real')
assume(z,'real')
assume(r,'real')
assume(t,'real')
assume(theta,'real')
assume(phi,'real')

In [ ]:
m1 = matrix([[0,1,0,0],
             [1,0,0,0],
             [0,0,1,0],
             [0,0,0,1]])

m2 = matrix([[0,1,0,0],
             [0,0,1,0],
             [1,0,0,0],
             [0,0,0,1]])

m3 = matrix([[0,1,0,0],
             [0,0,1,0],
             [0,0,0,1],
             [1,0,0,0]])

refl = matrix([[-1, 0, 0,0],
               [ 0,-1, 0, 0],
               [ 0, 0,-1, 0],
               [ 0, 0, 0,-1]])

In [ ]:
com = vector([1,1,1,1])/sqrt(4)

u = vector(QQ,[1,0,0,0])
v = vector(QQ,[0,1,0,0])
w = vector(QQ,[0,0,1,0])

u = u - (u*com)*com/com.norm()^2
v = v - (v*com)*com/com.norm()^2
w = w - (w*com)*com/com.norm()^2

v = v - (v*u)*u/u.norm()^2
w = w - (w*u)*u/u.norm()^2

w = w - (w*v)*v/v.norm()^2

u = u/u.norm()
v = v/v.norm()
w = w/w.norm()


In [ ]:
Pm = matrix([com,u,v,w]).transpose()

Qm = matrix([[0,0,0],
             [1,0,0],
             [0,1,0],
             [0,0,1]])

s1 = Qm.transpose()*Pm^-1*m1*Pm*Qm
s2 = Qm.transpose()*Pm^-1*m2*Pm*Qm
s3 = Qm.transpose()*Pm^-1*m3*Pm*Qm
srefl = Qm.transpose()*Pm^-1*refl*Pm*Qm

assert s1^2 == identity_matrix(3)
assert s2^3 == identity_matrix(3)
assert s3^4 == identity_matrix(3)

In [ ]:
S4 = [s1^i * s2^j * s3^k for (i,j,k) in product(range(2),range(3),range(4)) ]

S4ext = S4 + [ m*srefl for m in S4]

for m in S4ext:
    m.set_immutable()

print(len(set(m for m in S4)))
print(len(set(m for m in S4ext)))

In [ ]:
M1 = matrix([[1,-1,0,0],[0,1,-1,0]])
M2 = matrix([[1,-1,0,0],[0,1,0,-1]])
M3 = matrix([[1,0,-1,0],[0,0,1,-1]])
M4 = matrix([[0,1,-1,0],[0,0,1,-1]])

#pretty_print(M1,M2,M3,M4)

Pl = matrix([u,v,w]).transpose()

#pretty_print(Pl)

N1 = M1*Pl
N2 = M2*Pl
N3 = M3*Pl
N4 = M4*Pl

w1 = N1.right_kernel().basis_matrix()[0]
w2 = N2.right_kernel().basis_matrix()[0]
w3 = N3.right_kernel().basis_matrix()[0]
w4 = N4.right_kernel().basis_matrix()[0]

w1 = (w1/w1.norm()).simplify_full()
w2 = (w2/w2.norm()).simplify_full()
w3 = (w3/w3.norm()).simplify_full()
w4 = (w4/w4.norm()).simplify_full()

In [ ]:
def legendre_poly(n):
    return diff( (x^2-1)^n, x, n)/(2^n*factorial(n))

def assoc_legendre_poly(m,l):
    if m >= 0:
        return (-1)^m * (1-x^2)^(m/2) *diff(legendre_poly(l),x,m)
    else:
        return (-1)^m * factorial(l-m) /factorial(l+m) * assoc_legendre_poly(-m,l)

def real_sh(m,l):
    return r^(-1-l)*cos(m*phi)*assoc_legendre_poly(m,l)(x=cos(theta))

def imag_sh(m,l):
    return r^(-1-l)*sin(m*phi)*assoc_legendre_poly(m,l)(x=cos(theta))

In [ ]:
def S4ext_average(f):
    params = [ m^-1 * vector([x, y, z]) for m in S4ext ]
    f_ave = sum([ f(x=Nm[0],y=Nm[1],z=Nm[2]) for Nm in params ])
    return f_ave / len(S4ext)

In [ ]:
for l in range(13):
    print l, matrix([vector([real_sh(m,l)(r=1,theta=arccos(z),phi=atan2(y,x))(x=wi[0],y=wi[1],z=wi[2]).simplify()
                             for wi in [w1,-w1,w2,-w2,w3,-w3,w4,-w4]])
                     for m in range(-l+1,l)]).rank()

In [ ]:
matrix([[integrate(integrate(g*f,phi,-pi,pi)*sin(theta),theta,0,pi) 
         for f in [ real_sh(n,4)(r=1) for n in range(-3,4) ] ]
        for g in [ S4ext_average(real_sh(m,4)(r=1,theta=arccos(z),phi=atan2(y,x)))(x=sin(theta)*cos(phi),y=sin(theta)*sin(phi),z=cos(theta)) for m in range(-3,4) ] ])

In [ ]:
for mp in range(-3,4):
    g = S4ext_average(real_sh(mp,4)(r=1,theta=arccos(z),phi=atan2(y,x)))(x=sin(theta)*cos(phi),y=sin(theta)*sin(phi),z=cos(theta))
    print [ integrate(integrate(g*f,phi,-pi,pi)*sin(theta),theta,0,pi) for f in [ real_sh(m,4)(r=1) for m in range(-3,4) ]]

In [ ]:
for l in range(13):
    for m in range(-l+1,l):
        f_r = real_sh(m,l)(r=1,theta=arccos(z),phi=atan2(y,x)).simplify()
        g_r = lambda (x,y,z): f_r(x=x,y=y,z=z)
        val_r = sum([ g_r(mat^-1*w1) for mat in S4ext]).simplify_full()
        
        f_i = imag_sh(m,l)(r=1,theta=arccos(z),phi=atan2(y,x)).simplify()
        g_i = lambda (x,y,z): f_i(x=x,y=y,z=z)
        val_i = sum([ g_i(mat^-1*w1) for mat in S4ext]).simplify_full()
        
        print (m,l), val_r, val_i

In [ ]:
f0 = (real_sh(0,4)/(14/3)-imag_sh(2,4)/(560/3*sqrt(2)))(r=1,theta=arccos(z),phi=atan2(y,x)).simplify_full()
g0 = imag_sh(0,4)(r=1,theta=arccos(z),phi=atan2(y,x)).simplify_full()

print(f0)
print(g0)

In [ ]:
harmonic_r = 0
harmonic_i = 0
for m in S4ext:
    Nm = m^-1 * vector([x, y, z])
    #harmonic += det(Mm)*f0(x=Nm[0],y=Nm[1],z=Nm[2])
    harmonic_r += f0(x=Nm[0],y=Nm[1],z=Nm[2])
    harmonic_i += g0(x=Nm[0],y=Nm[1],z=Nm[2])
    
    
if harmonic_r == 0:
    print "harmonic_r==0"
    
if harmonic_i == 0:
    print "harmonic_i==0"
    
#print(harmonic_r)
#print(harmonic_i)

In [ ]:
for n, wi in enumerate([w1,-w1,w2,-w2,w3,-w3,w4,-w4]):
    val = harmonic_r(x=wi[0],y=wi[1],z=wi[2]).simplify()
    assert val == 0, (n, val)
    
for wi in [w1,-w1,w2,-w2,w3,-w3,w4,-w4]:
    val = harmonic_i(x=wi[0],y=wi[1],z=wi[2]).simplify()
    assert val == 0    

In [ ]:
wv_r = harmonic_r(x=sin(theta)*cos(phi),y=sin(theta)*sin(phi),z=cos(theta)).simplify_trig()
wv_i = harmonic_i(x=sin(theta)*cos(phi),y=sin(theta)*sin(phi),z=cos(theta)).simplify_trig()

In [ ]:
Pmag_r = density_plot(abs(wv_r), (theta,0,pi), (phi,-pi,pi), cmap=cm.viridis,
                    plot_points=200,aspect_ratio=1,axes=False)

Pmag_r.show()

In [ ]:
Pmag_i = density_plot(abs(wv_i), (theta,0,pi), (phi,-pi,pi), cmap=cm.viridis,
                    plot_points=200,aspect_ratio=1,axes=False)

Pmag_i.show()

In [ ]:
def stereographic_proj(v):
    """ Standard Stereographic projection with possible pre-rotation about X, Y and Z axis"""
    w = vector(v)
    return (w[0]/(1-w[2]),w[1]/(1-w[2]))

def to_iso_spherical_coords(v):
    x, y, z = v
    r = sqrt(x^2+y^2+z^2)
    theta = arccos(z/r)
    phi = atan2(y,x)
    return (r,theta,phi)

def to_iso_sphere_coords(v):
    r, theta, phi = to_iso_spherical_coords(v)
    return (theta,phi)

In [ ]:
P = Graphics()

var('t')

from itertools import combinations

# (-w2,-w3), (-w1,-w3)

# the 2-interations
for (wi, wj) in [(-w1,-w2),(-w1,-w3),(-w1,w4),(-w2,-w3),(-w2,w4),(-w3,w4)]:
    three_curve = cos(t)*wi+sin(t)*wj
    three_curve = three_curve/three_curve.norm()
    #two_curve = vector(stereographic_proj(three_curve)).simplify()
    two_curve = vector(to_iso_sphere_coords(three_curve)).simplify()
    if (wi == -w1 or wi == -w2) and wj == -w3:
        P += parametric_plot(two_curve,
                             (t,0,arctan(1/2)-0.01),
                             thickness=2,
                             color=rainbow(3)[0])
        P += parametric_plot(two_curve,
                             (t,arctan(1/2)+0.01,pi/2),
                             thickness=2,
                             color=rainbow(3)[0])
    else:
        P += parametric_plot(two_curve,
                             (t,0,pi/2),
                             thickness=2,
                             color=rainbow(3)[0])
        
    P += parametric_plot(two_curve,
                         (t,pi/2,pi),
                         thickness=2,
                         color=rainbow(3)[1])
    P += parametric_plot(two_curve,
                         (t,pi,3*pi/2),
                         thickness=2,
                         color=rainbow(3)[2])
    P += parametric_plot(two_curve,
                         (t,3*pi/2+0.01,2*pi-0.01),
                         thickness=2,
                         color=rainbow(3)[1])    


P += point(to_iso_sphere_coords( w1), size=120, color='black')
P[-1].set_zorder(10)
P += point(to_iso_sphere_coords(-w1), size=120, color='black')
P[-1].set_zorder(10)

P += point(to_iso_sphere_coords( w2), size=120, color='black')
P[-1].set_zorder(10)
P += point(to_iso_sphere_coords(-w2), size=120, color='black')
P[-1].set_zorder(10)

P += point(to_iso_sphere_coords( w3), size=120, color='black')
P[-1].set_zorder(10)
P += point(to_iso_sphere_coords(-w3), size=120, color='black')
P[-1].set_zorder(10)

P += point(to_iso_sphere_coords( w4), size=120, color='black')
P[-1].set_zorder(10)
P += point(to_iso_sphere_coords(-w4), size=120, color='black')
P[-1].set_zorder(10)

#P.show(aspect_ratio=1,axes=False)

In [ ]:
Q_r = P + Pmag_r
Q_r.show(aspect_ratio=1,axes=False, figsize=12)

In [ ]:
Q_r.save('/home/knappa/wv_r.pdf',aspect_ratio=1,axes=False, figsize=12)

In [ ]:
Q_i = P + Pmag_i
Q_i.show(aspect_ratio=1,axes=False, figsize=12)

In [ ]:
Q_i.save('/home/knappa/wv_i.pdf',aspect_ratio=1,axes=False, figsize=12)